# Carla 02

## Toubleshooting: Conda in VS code Python notebook 

Conda may not work in VS code Python notebook. Here are some common issues and solutions.

### Check if everything works

From a python notebook in VS code, click on `Select kernel` in the top right corner.

![Python notebook kernel selection](img/python-kernel.png)

Run a cell with the following code (install `ipykernel` if requested):

```python
import carla
```

### Conda not found in VS code but works in terminal

If you have installed conda and it works in terminal but not in VS code, you may need to add the conda path to the VS code settings.

1. Search for `conda.exe` absolute path in terminal. (e.g. `C:\anaconda3\Scripts\conda.exe`).
2. Open the VS code settings by pressing `Ctrl + ,` (or `File > Preferences > Settings`).
3. Search for `python.condaPath` in the search bar.
4. Paste the conda path in the `Python: Conda Path` field.
    ![Python conda path](img/python-conda-path.png)
5. Save the settings and restart VS code.


In [1]:
import carla, time, pygame, math, random

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

## Some helpful functions used in this notebook

In [3]:
def move_spectator_to(transform, spectator, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(world, vehicle_index=0, spawn_index=0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content="O", color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)


## Making the Spectator Follow a Driving Car

In this exercise, we will use the CARLA simulator to spawn a vehicle and make it drive autonomously using the autopilot feature. Additionally, we will set the spectator's view to follow the vehicle from above, providing a bird's-eye view of the car as it navigates through the environment.

### Steps:
1. Spawn a vehicle
2. Set the vehicle's autopilot mode
3. Make the spectator follows the vehicle

In [4]:
vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)

while True:
    move_spectator_to(vehicle.get_transform(), spectator)
    world.tick()

KeyboardInterrupt: 

In [5]:
vehicle.destroy()

True

## Waypoint

Waypoints are specific points along the road network that help define the driving path for vehicles. They are essential for navigation, as they represent locations on the road where vehicles can move to while following a specific route. Waypoints linked together form a continuous path that autonomous agents, like cars or pedestrians, can follow to move smoothly across the environment.

![waypoints](img/waypoints.png)

### Example of waypoint creation

Waypoints can be created manually or automatically using the CARLA Python API. In the example below, we create waypoints based on car position.

In [6]:
vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)

waypoints_list = []

for _ in range(10):
    transform = vehicle.get_transform()
    waypoint = world.get_map().get_waypoint(transform.location)
    waypoints_list.append(waypoint)
    world.tick()
    time.sleep(1)

for wp in waypoints_list:
    print(wp.transform.location)

Location(x=-0.036634, y=13.183878, z=0.000000)
Location(x=-64.373138, y=24.505157, z=0.000000)
Location(x=-60.946144, y=24.242323, z=0.000000)
Location(x=-54.560444, y=21.849924, z=0.000000)
Location(x=-48.981083, y=16.547897, z=0.000000)
Location(x=-45.516319, y=9.464963, z=0.000000)
Location(x=-45.302486, y=1.421034, z=0.000000)
Location(x=-45.324566, y=-6.424444, z=0.000000)
Location(x=-45.353333, y=-13.989666, z=0.000000)
Location(x=-45.305828, y=-21.721939, z=0.000000)


In [7]:
vehicle.destroy()

True

### Debug utily: draw something in the screen

In order to help visualize positions and debug the code, we can draw points, lines, and other shapes directly on the screen. This can be useful for understanding the vehicle's position, orientation, trajectory, and so on. These particular elements can also be automatically removed from the screen after a certain amount of time, making it easier to track changes in the simulation.

Available shapes are arrow, box, line, point, and string. Here methods used to draw them:

- `world.debug.draw_arrow`
- `world.debug.draw_box`
- `world.debug.draw_line`
- `world.debug.draw_point`
- `world.debug.draw_string`

In [9]:
for wp in waypoints_list:
    world.debug.draw_string(
        wp.transform.location,
        'Waypoint Here!',
        color=carla.Color(r=0, g=255, b=0),
        life_time=20.0, # life_time=0 means infinite
    )

### Roads

Roads are the main infrastructure in the CARLA simulator, representing the paths where vehicles can drive. They are composed of lanes, waypoints, and other elements that define the driving environment. Roads can have different shapes, sizes, and configurations, depending on the specific scenario or map being used.

Now we will explore roads as a sequence of waypoints, plotting them on the screen to visualize the driving path that a vehicle could follow.

In [10]:
roads = world.get_map().get_topology()

waypoint = roads[0][0]
draw_on_screen(world, waypoint.transform)

In [11]:
for i in range(1000):
    # get the next waypoint with distance 1
    waypoint = waypoint.next(1)[0]
    draw_on_screen(world, waypoint.transform)
    time.sleep(0.01)

Now let's drawing some **roads** in the screen.

In [12]:
def color_generator():
    colors = [
        carla.Color(r=r, g=g, b=b) for r in [0, 255] for g in [0, 255] for b in [0, 255]
    ]
    while True:
        for color in colors:
            yield color

for i, color in zip(range(len(roads)), color_generator()):
    waypoint = roads[i][0]
    for j in range(1000):
        waypoint = waypoint.next(3)[0]
        draw_on_screen(world, waypoint.transform, color=color, life_time=10)
    time.sleep(0.5)

KeyboardInterrupt: 

## Positioning vs Controlling a Vehicle

Now, we will explore the difference between positioning and controlling a vehicle.

- **Positioning**: setting the vehicle's location and orientation in the environment
- **Controlling**: managing the vehicle's speed, steering, and other driving parameters.

### Position a Vehicle

Position a vehicle is useful when we want to place a vehicle at a specific location in the environment. This can be considered a static operation, as the vehicle will not move unless explicitly controlled.

In [13]:
vehicle = spawn_vehicle(world)
roads = world.get_map().get_topology()
waypoint = roads[0][0]

vehicle.set_transform(waypoint.transform)

While positioning a vehicle, with tuning of the timing and the location, could result like the vehicle is moving in the environment, this is just a visual effect, as the vehicle is not actively controlled.

This operation **cannot be considered as driving**, as the vehicle is not moving autonomously or following a specific path.

In [14]:
sleep_time = 1
waypoint_distance = 1

try:
    while True:
        waypoint = waypoint.next(waypoint_distance)[0]
        vehicle.set_transform(waypoint.transform)
        time.sleep(sleep_time)
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()

### Control a Vehicle

Controlling a vehicle is useful when we want to make a vehicle move in the environment.

This can be considered a dynamic operation, as the vehicle will change its position and orientation based on the control commands given.

In [17]:
vehicle = spawn_vehicle(world)

throttle = 0.6

def move_forward(vehicle, duration):
    control = carla.VehicleControl()
    control.throttle = throttle
    vehicle.apply_control(control)
    start_time = time.time()
    while time.time() - start_time < duration:
        world.tick()
        time.sleep(0.1)

def steer_left(vehicle):
    control = carla.VehicleControl()
    control.throttle = throttle
    control.steer = -0.38
    vehicle.apply_control(control)
    world.tick()
    time.sleep(2)

try:
    move_forward(vehicle, 5)
    steer_left(vehicle)
    move_forward(vehicle, 20)
finally:
    vehicle.destroy()

### Example of random movement

In [18]:
vehicle = spawn_vehicle(world)

def random_control(vehicle):
    control = carla.VehicleControl()
    control.throttle = random.uniform(0.5, 1.0)
    control.steer = random.uniform(-1.0, 1.0)
    vehicle.apply_control(control)

try:
    while True:
        random_control(vehicle)
        world.tick()
        time.sleep(0.1)
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()

## Exercise 1: looking reverse

Based on the spectator that follows the car code, make the it look in the opposite direction when a key is pressed (e.g., `r`).

To get user input, you can use `pygame` library with the following code. 

In [20]:
vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)



pygame.init()
# focus on the pygame window to get the key pressed
pygame.display.set_mode((400, 300))



running = True
reverse_view = False
while running:
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            key = event.key
            print(f"Key pressed: {pygame.key.name(key)}")
            if key == pygame.QUIT or key == pygame.K_ESCAPE:
                running = False
            elif key == pygame.K_r:
                reverse_view = not reverse_view
    move_spectator_to(vehicle.get_transform(), spectator, yaw=180 if reverse_view else 0)
    world.tick()
    pygame.display.flip()

pygame.quit()

Key pressed: r
Key pressed: r
Key pressed: r
Key pressed: r
Key pressed: r
Key pressed: r


KeyboardInterrupt: 

### Solution

In [ ]:
pygame.init()
screen = pygame.display.set_mode((400, 300))

vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)

running = True
reverse_view = False
while running:
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            key = event.key
            if key == pygame.QUIT or key == pygame.K_ESCAPE:
                running = False
            elif key == pygame.K_r:
                reverse_view = not reverse_view

    move_spectator_to(vehicle.get_transform(), spectator, yaw=180 if reverse_view else 0)
    world.tick()
    pygame.display.flip()

pygame.quit()
vehicle.destroy()

## Exercise 2: draw a debug circle around a car

Based on the code that draws a debug circle around a car, make the circle follow the car as it moves.

### Solution

In [21]:
vehicle = spawn_vehicle(world)
vehicle.set_autopilot(True)

try:
    while True:
        transform = vehicle.get_transform()

        # draw a circle around the vehicle
        for i in range(0, 360, 36):
            angle = math.radians(i)
            x = transform.location.x + math.cos(angle)
            y = transform.location.y + math.sin(angle)
            circle_location = carla.Location(x=x, y=y, z=transform.location.z)
            circle_transform = carla.Transform(circle_location, transform.rotation)
            draw_on_screen(world, circle_transform, color=carla.Color(255, 0, 0), life_time=0.1)

        world.tick()
        time.sleep(0.1)
finally:
    vehicle.destroy()

KeyboardInterrupt: 

## Exercise 3: Controlled Car

Make the car move forward when the `w` key is pressed and stop when the `s` key is pressed.

### Solution

In [22]:
pygame.init()
screen = pygame.display.set_mode((400, 300))

vehicle = spawn_vehicle(world)
time.sleep(1)
move_spectator_to(vehicle.get_transform(), spectator)

running = True
moving_forward = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            key = event.key
            if key == pygame.QUIT or key == pygame.K_ESCAPE:
                running = False
            elif key == pygame.K_w:
                moving_forward = True
            elif key == pygame.K_s:
                moving_forward = False

    if moving_forward:
        control = carla.VehicleControl(throttle=0.5)
    else:
        control = carla.VehicleControl(throttle=0.0)

    vehicle.apply_control(control)
    world.tick()
    pygame.display.flip()

pygame.quit()
vehicle.destroy()

KeyboardInterrupt: 

## Exercise 3 bis: Pattern based movement

Make the car move following specific patterns.

- when the `z` key is pressed, the car should move following a **zigzag** pattern.
- when the `x` key is pressed, the car should move following a **square** pattern.
- when the `c` key is pressed, the car should move following a **circle** pattern.

### Solution

In [ ]:
pygame.init()
screen = pygame.display.set_mode((400, 300))

vehicle = spawn_vehicle(world)
time.sleep(1)
move_spectator_to(vehicle.get_transform(), spectator)

running = True
pattern = None

def zigzag(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.5
    for _ in range(10):
        control.steer = 0.5
        vehicle.apply_control(control)
        world.tick()
        time.sleep(1)
        control.steer = -0.5
        vehicle.apply_control(control)
        world.tick()
        time.sleep(1)

def square(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.5
    vehicle.apply_control(control)
    time.sleep(7)
    for _ in range(4):
        control.steer = 0
        vehicle.apply_control(control)
        world.tick()
        time.sleep(2)
        control.steer = -1
        vehicle.apply_control(control)
        world.tick()
        time.sleep(1.55)

def circle(vehicle):
    control = carla.VehicleControl()
    control.throttle = 0.7
    vehicle.apply_control(control)
    time.sleep(4)
    control.steer = -1
    vehicle.apply_control(control)
    world.tick()
    time.sleep(10)

while running:
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            key = event.key
            if key == pygame.QUIT or key == pygame.K_ESCAPE:
                running = False
            elif key == pygame.K_z:
                pattern = 'zigzag'
            elif key == pygame.K_x:
                pattern = 'square'
            elif key == pygame.K_c:
                pattern = 'circle'

    if pattern == 'zigzag':
        zigzag(vehicle)
        pattern = None
    elif pattern == 'square':
        square(vehicle)
        pattern = None
    elif pattern == 'circle':
        circle(vehicle)
        pattern = None
    
    control = carla.VehicleControl()
    control.steer = 0
    control.throttle = 0
    vehicle.apply_control(control)

    pygame.display.flip()

pygame.quit()
vehicle.destroy()

RuntimeError: Spawn failed because of collision at spawn position

: 

## Exercise 3 tris: Pacman Effect

Make the car move straightforward and when it reaches the end of the road, the so-called "Pacman effect" should happen, i.e., the car should appear on the opposite side of the world.

[Pacman Effect](https://en.wiktionary.org/wiki/Pac-Man_effect)

In [ ]:
vehicle = spawn_vehicle(world)

ROAD_X_MIN = -110
ROAD_X_MAX = 110

try:
    while True:
        transform = vehicle.get_transform()
        location = transform.location

        print(f"Location: {location.x}, {location.y}", end='\r')

        if location.x > ROAD_X_MAX:
            location.x = ROAD_X_MIN
            vehicle.set_transform(carla.Transform(location, transform.rotation))

        control = carla.VehicleControl(throttle=0.5)
        vehicle.apply_control(control)

        world.tick()
        time.sleep(0.1)
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()